In [7]:
from flask import Blueprint, render_template
from flask import Flask, redirect, url_for, render_template
from flask import Flask, render_template, session, request, redirect, jsonify, g, url_for, request, make_response
import spacy
import nltk
import re
import pprint
import pandas as pd
import pdb
from nltk.stem import WordNetLemmatizer     
from pathlib import Path
import os
from nltk import Tree
import pdb
import inflect
import wordninja
from itertools import groupby
from nltk.tokenize import word_tokenize
import numpy as np
import inflect
import shutil
from os import path
import os, shutil, glob

In [8]:
with open(r"input.txt", "r") as text:
    text = text.read()
    #print(FileContent)

In [9]:
def transcriber():
        global text
        se=text
        nlp = spacy.load("en_core_web_sm")
        
        #####################################READ THE FILE WITH THE BELOW CODE#########################################################
        q8=''
        q1=[]
        q7=''   
        inputdataa=se.lower()     


        ###################################remove a person name or a location in the sentance using ner from spacy###################################################################    

        doc = nlp(inputdataa)
        persons = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
        doc_anon = nlp(" ".join([d for d in doc.text.split() if d not in persons]))
        inptdata=''.join([token.text_with_ws for token in doc_anon])
        doc = nlp(inptdata)
        persons = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
        doc_ano = nlp(" ".join([d for d in doc.text.split() if d not in persons]))
        #print(doc_ano)   
        inputdataas=''.join([token.text_with_ws for token in doc_ano])                  
        inputdata=inputdataas

        #########################We add the word 'yes' after every comma and fullstop and delete them later so that they are not confused as a part of an integer. They will be more efficient when they pass through the word to number function#################################################################################3
        rx = r"\."
        mem = re.sub(rx, ". yes ", inputdata)
        ry = r"\,"
        ina = re.sub(ry, ", yes ", mem)
        rz = r" "
        inputdata = re.sub(rz, " yes ", ina)
        ########################convert word to number##################################

        def is_number(x):
            if type(x) == str:
                x = x.replace('', '')
            try:
                float(x)
            except:
                return False
            return True

        def text2int (textnum, numwords={}):
            units = [
                'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
                'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen',
                'sixteen', 'seventeen', 'eighteen', 'nineteen',
            ]
            tens = ['', '', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
            scales = ['hundred', 'thousand', 'million', 'billion', 'trillion']
            ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
            ordinal_endings = [('ieth', 'y'), ('th', '')]

            if not numwords:
                numwords['and'] = (1, 0)
                for idx, word in enumerate(units): numwords[word] = (1, idx)
                for idx, word in enumerate(tens): numwords[word] = (1, idx * 10)
                for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)


            current = result = 0
            curstring = ''
            onnumber = False
            lastunit = False
            lastscale = False

            def is_numword(x):
                if is_number(x):
                    return True
                if word in numwords:
                    return True
                return False

            def from_numword(x):
                if is_number(x):
                    scale = 0
                    increment = float(x.replace(',', ''))
                    return scale, increment
                return numwords[x]

            for word in textnum.split():
                if word in ordinal_words:
                    scale, increment = (1, ordinal_words[word])
                    current = current * scale + increment
                    if scale > 100:
                        result += current
                        current = 0
                    onnumber = True
                    lastunit = False
                    lastscale = False
                else:
                    for ending, replacement in ordinal_endings:
                        if word.endswith(ending):
                            word = "%s%s" % (word[:-len(ending)], replacement)

                    if (not is_numword(word)) or (word == 'and' and not lastscale):
                        if onnumber:
                            # Flush the current number we are building
                            curstring += repr(result + current) + " "
                        curstring += word + " "
                        result = current = 0
                        onnumber = False
                        lastunit = False
                        lastscale = False
                    else:
                        scale, increment = from_numword(word)
                        onnumber = True

                        if lastunit and (word not in scales):                                                                                                                                                                                                                                         
                            # Assume this is part of a string of individual numbers to                                                                                                                                                                                                                
                            # be flushed, such as a zipcode "one two three four five"                                                                                                                                                                                                                 
                            curstring += repr(result + current)                                                                                                                                                                                                                                       
                            result = current = 0                                                                                                                                                                                                                                                      

                        if scale > 1:                                                                                                                                                                                                                                                                 
                            current = max(1, current)                                                                                                                                                                                                                                                 

                        current = current * scale + increment                                                                                                                                                                                                                                         
                        if scale > 100:                                                                                                                                                                                                                                                               
                            result += current                                                                                                                                                                                                                                                         
                            current = 0                                                                                                                                                                                                                                                               

                        lastscale = False                                                                                                                                                                                                              
                        lastunit = False                                                                                                                                                
                        if word in scales:                                                                                                                                                                                                             
                            lastscale = True                                                                                                                                                                                                         
                        elif word in units:                                                                                                                                                                                                             
                            lastunit = True

            if onnumber:
                curstring += repr(result + current)

            return curstring
        texta=text2int(inputdata)

        text=texta.replace('yes', ' ')
        ##############################text is the string that contains integers############################################################################
        r1=text.split(' ')
        for i1 in range (len(r1)):
            if r1[i1]=='half' or r1[i1]=='half.' or r1[i1]=='half,':
                r1[i1]='0.5'

        for z1 in range (len(r1)):
            if r1[z1]=='a':
                r1[z1]='1.0'


        for z3 in range (len(r1)):
            if r1[z3]=='for' or r1[z3]=='for,' or r1[z3]=='for.':
                r1[z3]='4.0'


        for i2 in range (len(r1)):
            if r1[i2]=='quarter' or r1[i2]=='quarter,' or r1[i2]=='quarter.':
                r1[i2]='0.25'

        for i3 in range (len(r1)):
            if r1[i3]=='kg' or r1[i3]=='kilos' or r1[i3]=='kilo' or r1[i3]=='kgs' or r1[i3]=='kilograms' or r1[i3]=='kg,' or r1[i3]=='kilos,' or r1[i3]=='kilo,' or r1[i3]=='kgs,' or r1[i3]=='kilograms,' or r1[i3]=='kg.' or r1[i3]=='kilos.' or r1[i3]=='kilo.' or r1[i3]=='kgs.' or r1[i3]=='kilograms.':                           
                r1[i3]='kilogram'                                                                                                                                                                                                                                                                                  

        for i4 in range (len(r1)):
            if r1[i4]=='l' or r1[i4]=='litres' or r1[i4]=='liter' or r1[i4]=='liters' or r1[i4]=='l.' or r1[i4]=='litres.' or r1[i4]=='liter.' or r1[i4]=='liters.' or r1[i4]=='l,' or r1[i4]=='litres,' or r1[i4]=='liter,' or r1[i4]=='liters,':
                r1[i4]='litre'
        for i5 in range (len(r1)):
            if r1[i5]=='g' or r1[i5]=='gram' or r1[i5]=='gs' or r1[i5]=='g,' or r1[i5]=='gram,' or r1[i5]=='gs,' or r1[i5]=='g.' or r1[i5]=='gram.' or r1[i5]=='gs.' or r1[i5]=='grams.' or r1[i5]=='grams' or r1[i5]=='grams,' :
                r1[i5]='gram'

        for i6 in range (len(r1)):
            if r1[i6]=='gallons' or r1[i6]=='gal' or r1[i6]=='gals' or r1[i6]=='gallons,' or r1[i6]=='gal,' or r1[i6]=='gals,' or r1[i6]=='gallons.' or r1[i6]=='gal.' or r1[i6]=='gals.'  :
                r1[i6]='gallon'

        for i7 in range (len(r1)):
            if r1[i7]=='lb' or r1[i7]=='pounds' or r1[i7]=='lbs' or r1[i7]=='lb,' or r1[i7]=='pounds,' or r1[i7]=='lbs,' or r1[i7]=='lb.' or r1[i7]=='pounds.' or r1[i7]=='lbs.' :
                r1[i7]='pounds'

        List_of_unwanted_word=['|','hi','hi,','hi.','hello','hello,','hello.','%','thanks,','give','batch','jars','jars,','jars.','jar.','jar,','jar,','thanks','thanks.', 'vat', 'bottle','batch,', 'vat,', 'bottle,','batch.', 'vat.', 'bottle.','box,','boxes,','box.','boxes.','box','boxes','batchs', 'vats', 'bottles','batchs,', 'vats,', 'bottles,','container','container.','container,','containers','containers.','containers,',"place",'batch', 'vat', 'bottle','batch,', 'vat,', 'bottle,','batch.', 'vat.', 'bottle.','box,','boxs,','box.','boxs.','box','boxs','batchs', 'vats', 'bottles','batchs,', 'vats,', 'bottles,','container','container.','container,','containers','containers.','containers,','can,','can.,','can','cans.','cans,','cans','of.','lot.','name.', 'coupon.','greeting.','discount.',"please.",'bag.','bags','bags.','bags,','order.','packet.','packets.','packets','packets,','time.', 'afternoon.','thank.','dinner.','night.','bottle.','bottles.','bottles','bottles,','evening.', 'please.' ,'morning.','great.','i.','hey.','breakfast.','pardon.','lunch.','pack.','packs','packs,','packs.','jar.' ,'of,','lot,','name,', 'coupon,','greeting,','discount,',"please,",'bag,','order,','packet,','time,', 'afternoon,','thank,','dinner,','night,','bottle,','evening,', 'please,' ,'morning,','great,','i,','hey,','breakfast,','pardon,','lunch,','pack,','jar,','of','lot','name', 'coupon','greeting','discount',"please",'bag','order','packet','time', 'afternoon','thank','dinner','night','bottle','evening', 'please' ,'morning','great','i','hey','breakfast','pardon','lunch','pack','jar']

        #############################List_of_unwanted_word is the custom list of stop words###########################################################################



        r2 = [item for item in r1 if item not in List_of_unwanted_word]        
        Sentance = ' '.join([str(elem) for elem in r2])

        ###################################"Sentance" contains only words that we want to analyse#######################################################################
        ##################################Now we use NLP to get all integers from "Senance"######################################################################
      
        pattern="""

                  NP:  {<CD>}

            """

        NPChunker = nltk.RegexpParser(pattern)

        def prepare_text(input):
            sentences = nltk.sent_tokenize(input)
            sentences = [nltk.word_tokenize(sent) for sent in sentences] 
            sentences = [nltk.pos_tag(sent) for sent in sentences]
            sentences = [NPChunker.parse(sent) for sent in sentences]
            return sentences


        def parsed_text_to_NP(sentences):
            nps = []
            for sent in sentences:
                tree = NPChunker.parse(sent)
               # print(tree)
                for subtree in tree.subtrees():
                    if subtree.label() == 'NP':
                        t = subtree
                        t = ' '.join(word for word, tag in t.leaves())
                        nps.append(t)
            return nps


        def sent_parse(input):
            sentences = prepare_text(input)
            nps = parsed_text_to_NP(sentences)
            return nps



        numberslis=    sent_parse(Sentance)       # We get a list of all numbers in the text

        numberslist=numberslis
        r12 = [itenm for itenm in r2 if itenm not in numberslist] 

        Sentancez = ' '.join([str(elems) for elems in r12])

        ################################We do NLP to find all the required noun phrases in the text######################################################################


        patterns="""
            NP:     
                    {<CD><NNS><NNS><NNS>}
                    {<CD><NNS><NNS><NN>}
                    {<CD><NNS><NN><NN>}
                    {<CD><FW><JJ><NN>}
                    {<CD><FW><JJ><NNS>}
                    {<CD><FW><NNS><NN>}
                    {<CD><FW><NN><NNS>}
                    {<CD><FW><NNS><NN>}
                    {<CD><JJR><NNS><NN>}
                    {<CD><JJR><NNS><NNS>}
                    {<CD><JJR><NN><NNS>}
                    {<CD><JJR><NN><NN>}
                    {<CD><JJR><NNS><NNS>}
                    {<CD><JJR><NN><NN>}
                    {<CD><JJR><NNS><NN>}
                    {<CD><JJR><NN><NNS>}
                    {<CD><JJR><JJ><NN>}
                    {<CD><JJR><JJ><NNS>}
                    {<CD><JJ><JJ><NNS>}
                    {<CD><JJ><JJ><NN>}
                    {<CD><NNS><NN><NN>}
                    {<CD><NNS><RBR><NNS>}
                    {<CD><NNS><NN><NNS>}
                    {<CD><NN><RBR><NN>}
                    {<CD><NNS><RBR><NN>}
                    {<CD><NN><RBR><NNS>}
                    {<CD><NN><JJ><NNS>}
                    {<CD><NN><JJ><NN>}
                    {<CD><NN><NNS><NNS>}
                    {<CD><NN><NNS><NN>}
                    {<CD><NNS><NN><NN>}
                    {<CD><NNS><NN><RB>}
                    {<CD><RBR><NNS><NN>}
                    {<CD><RBR><NN><NN>}
                    {<CD><RBR><NNS><NNS>}
                    {<CD><RBR><NN><NNS>}
                    {<CD><JJ><JJ>}
                    {<CD><NNS><JJ>}
                    {<CD><NN><JJ>}
                    {<CD><NN><VBP>}
                    {<CD><NNS><VBP>}  
                    {<CD><VBP><JJ>}
                    {<CD><JJ><NN>}
                    {<CD><JJ><NNS>}
                    {<CD><VBZ><NN>}
                    {<CD><VBZ><NN>}
                    {<CD><VB><NN>}
                    {<CD><VBP><NN>}
                    {<CD><VBP><NNS>}
                    {<CD><VBZ><JJ>}
                    {<CD><VBP><JJR>}
                    {<CD><VBP><FW>}
                    {<CD><VBR><NN>}
                    {<CD><RBR><NN>}
                    {<CD><JJR><NNS>}
                    {<CD><JJR><NN>}
                    {<CD><FW><NNS>}
                    {<CD><FW><NN>}
                    {<CD><NN><NN>}
                    {<CD><NNS><NN>}
                    {<CD><NN><NNS>}
                    {<CD><NN><NN>}
                    {<CD><NNS><NN>}
                    {<CD><NN><NNS>}
                    {<CD><NNS><NNS>}
                    {<CD><NNS><RBR>}
                    {<CD><NN><RBR>}
                    {<CD><NNS><RBR>}
                    {<CD><RBR><NN>}
                    {<CD><RBR><NNS>} 
                    {<CD><NN>}
                    {<CD><NNS>}
                    {<CD><VBZ>}
                    {<CD><VBP>}
                    {<CD><JJ>}
                    {<NNS><NNS><CD><NNS>}
                    {<NNS><NNS><CD><NN>}
                    {<FW><JJ><CD><NN>}
                    {<FW><JJ><CD><NNS>}
                    {<FW><NNS><CD><NN>}
                    {<FW><NN><CD><NNS>}
                    {<FW><NNS><CD><NN>}
                    {<JJR><NNS><CD><NN>}
                    {<JJR><NNS><CD><NNS>}
                    {<JJR><NN><CD><NNS>}
                    {<JJR><NN><CD><NN>}
                    {<JJR><NNS><CD><NNS>}
                    {<JJR><NN><CD><NN>}
                    {<JJR><NNS><CD><NN>}
                    {<JJ><NNS><CD><NN>}
                    {<JJR><NN><CD><NNS>}
                    {<JJR><JJ><CD><NN>}
                    {<JJR><JJ><CD><NNS>}
                    {<JJ><JJ><CD><NNS>}
                    {<JJ><JJ><CD><NN>}
                    {<NNS><RBR><CD><NNS>}
                    {<NN><RBR><CD><NN>}
                    {<NNS><RBR><CD><NN>}
                    {<NN><RBR><CD><NNS>}
                    {<NN><JJ><CD><NNS>}
                    {<NN><JJ><CD><NN>}
                    {<VBP><CD><NNS>}
                    {<NNS><CD><NN>}
                    {<JJ><CD><JJ>}
                    {<NNS><CD><JJ>}
                    {<NN><CD><JJ>}
                    {<NN><CD><NNS>}
                    {<NN><CD><NN>}
                    {<NN><CD><VBP>}
                    {<NNS><CD><VBP>}  
                    {<VBP><CD><JJ>}
                    {<VBZ><CD><NN>}
                    {<VBZ><CD><NN>}
                    {<VBP><CD><NN>}
                    {<VBP><CD><NNS>}
                    {<NN><NNS><CD><NN>}
                    {<RBR><NNS><CD><NN>}
                    {<RBR><NN><CD><NN>}
                    {<RBR><NNS><CD><NNS>}
                    {<RBR><NN><CD><NNS>} 
                    {<NNS><NNS><NNS><CD>}
                    {<NNS><NNS><NN><CD>}
                    {<NNS><NN><NN><CD>}
                    {<FW><JJ><NN><CD>}
                    {<FW><JJ><NNS><CD>}
                    {<FW><NNS><NN><CD>}
                    {<FW><NN><NNS><CD>}
                    {<FW><NNS><NN><CD>}
                    {<JJR><NNS><NN><CD>}
                    {<JJR><NNS><NNS><CD>}
                    {<JJR><NN><NNS><CD>}
                    {<JJR><NN><NN><CD>}
                    {<JJR><NNS><NNS><CD>}
                    {<JJR><NN><NN><CD>}
                    {<JJR><NNS><NN><CD>}
                    {<JJR><NN><NNS><CD>}
                    {<JJR><JJ><NN><CD>}
                    {<JJR><JJ><NNS><CD>}
                    {<JJ><JJ><NNS><CD>}
                    {<JJ><JJ><NN><CD>}
                    {<NNS><NN><NN><CD>}
                    {<NNS><RBR><NNS><CD>}
                    {<NN><RBR><NN><CD>}
                    {<NNS><RBR><NN><CD>}
                    {<NN><RBR><NNS><CD>}
                    {<NN><JJ><NNS><CD>}
                    {<NN><JJ><NN><CD>}
                    {<NN><NNS><NNS><CD>}
                    {<NN><NNS><NN><CD>}
                    {<NNS><NN><NN><CD>}
                    {<RBR><NNS><NN><CD>}
                    {<RBR><NN><NN><CD>}
                    {<RBR><NNS><NNS><CD>}
                    {<RBR><NN><NNS><CD>}
                    {<NN><NNS><CD>}
                    {<NN><NN><CD>}
                    {<NNS><NN><CD>}
                    {<NN><NNS><CD>}
                    {<VBZ><JJ><CD>}
                    {<VBP><JJR><CD>}
                    {<NNS><CD><NNS>}
                    {<VBP><FW><CD>}
                    {<VBR><NN><CD>}
                    {<RBR><NN><CD>}
                    {<JJR><NNS><CD>}
                    {<JJR><NN><CD>}
                    {<FW><NNS><CD>}
                    {<FW><NN><CD>}
                    {<NN><NN><CD>}
                    {<NNS><NN><CD>}
                    {<NN><VBD><CD>}
                    {<NNS><VBD><CD>}
                    {<VB><NN><CD>}
                    {<NNS><RBR><CD>}
                    {<NN><RBR><CD>}
                    {<NNS><RBR><CD>}
                    {<RBR><NN><CD>}
                    {<RBR><NNS><CD>}
                    {<RBR><CD>}
                    {<NN><CD>}
                    {<NNS><CD>}
                    {<VBZ><CD>}
                    {<VBP><CD>}
                    {<VBD><CD>}
                    {<JJ><CD>}
                    {<RBR><CD>}
                    {<RB><CD>}

            """

        NPChunker = nltk.RegexpParser(patterns)

        def prepare_text(input):
            sentences = nltk.sent_tokenize(input)
            sentences = [nltk.word_tokenize(sent) for sent in sentences] 
            sentences = [nltk.pos_tag(sent) for sent in sentences]
            sentences = [NPChunker.parse(sent) for sent in sentences]
            return sentences


        def parsed_text_to_NP(sentences):
            nps = []
            for sent in sentences:
                tree = NPChunker.parse(sent)
               # print(tree)
                for subtree in tree.subtrees():
                    if subtree.label() == 'NP':
                        t = subtree
                        t = ' '.join(word for word, tag in t.leaves())
                        nps.append(t)
            return nps


        def sent_parse(input):
            sentences = prepare_text(input)
            nps = parsed_text_to_NP(sentences)
            return nps



        #if __name__ == '__main__':
         #   print(sent_parse(Sentance))

        pop=    sent_parse(Sentance)

        ##################################"items" contain a list of noun phrases that contain a grocery item (1kg of rice)################################################################################################
        p=[]
        wordz=[]
        for phrase in pop:
            words= phrase.split(' ')
            wordz.append(words)
            #words.pop(0)
            #print(words)


            for o in words:

                if o=='pound' or o== 'litre' or  o== 'kilogram' or o=='gram' or o=='gallon' :
                    #print('yes')
                    #print(words)
                    words.append(words.pop(words.index(o)))
         #   print(words)
            p.append(words)
                    #some_list = ['abc-123', 'def-456', 'ghi-789', 'abc-456']
                  #  print(o)
                  #  words.insert(-1, o)

        #print( p)

        y=[]
        for iy in p:
            if iy[-1]=='kilogram' or iy[-1]=='gram' or iy[-1]=='litre' or iy[-1]=='gallon' or iy[-1]=='pound':

                y.append(iy[-1])
            else:
                y.append('--')
        yu=[]
        #print(y)
        iop=[]
        for jj in p:
            if jj[-1]=='kilogram' or jj[-1]=='gram' or jj[-1]=='litre' or jj[-1]=='gallon' or jj[-1]=='pound':
                jj.pop()
                iop.append(jj)
        #print(wordz)
        mylist=y
        #numberslist=re.findall(r'[\d\.\d]+',Sentance )
        #for ia in numberslist:
        #    if ia=='.':
        #        numberslist.remove(ia)

        itemswoquants=p


        try:
            pup=[]
            for i in numberslist:
                if len(i)>1:
                    pup.append(w2n.word_to_num(i))
                else:
                    pup.append(i)

            numberslist=pup
        except Exception:
            pass

        #print('itemz')
        #print(itemswoquants)
        #itemssentance=''.join(itemswoquants)
        #print(itemssentance)


        #r2 = [item for item in r1 if item not in List_of_unwanted_word]  
        itementance = [red for red in r2 if red not in mylist]
        #itemssentance=' '.join(li)
        itemssentance=' '.join(itementance)
        #print(itemssentance)

        #################Here we pass the sentance that does not contain the units of measurement in order to obtain the nouns and noun phrases that are only grocery items####################################################################################################

        patternz="""

                  NP:  

                    {<CD><NNS><NNS><NNS>}
                    {<CD><NNS><NNS><NN>}
                    {<CD><NNS><NN><NN>}
                    {<CD><FW><JJ><NN>}
                    {<CD><FW><JJ><NNS>}
                    {<CD><FW><NNS><NN>}
                    {<CD><FW><NN><NNS>}
                    {<CD><FW><NNS><NN>}
                    {<CD><JJR><NNS><NN>}
                    {<CD><JJR><NNS><NNS>}
                    {<CD><JJR><NN><NNS>}
                    {<CD><JJR><NN><NN>}
                    {<CD><JJR><NNS><NNS>}
                    {<CD><JJR><NN><NN>}
                    {<CD><JJR><NNS><NN>}
                    {<CD><JJR><NN><NNS>}
                    {<CD><JJR><JJ><NN>}
                    {<CD><JJR><JJ><NNS>}
                    {<CD><JJ><JJ><NNS>}
                    {<CD><JJ><JJ><NN>}
                    {<CD><NNS><NN><NN>}
                    {<CD><NNS><RBR><NNS>}
                    {<CD><NNS><NN><NNS>}
                    {<CD><NN><RBR><NN>}
                    {<CD><NNS><RBR><NN>}
                    {<CD><NN><RBR><NNS>}
                    {<CD><NN><JJ><NNS>}
                    {<CD><NN><JJ><NN>}
                    {<CD><NN><NNS><NNS>}
                    {<CD><NN><NNS><NN>}
                    {<CD><NNS><NN><NN>}
                    {<CD><NNS><NN><RB>}
                    {<CD><RBR><NNS><NN>}
                    {<CD><RBR><NN><NN>}
                    {<CD><RBR><NNS><NNS>}
                    {<CD><RBR><NN><NNS>}
                    {<NNS><NNS><NNS><CD>}
                    {<NNS><NNS><NN><CD>}
                    {<NNS><NN><NN><CD>}
                    {<FW><JJ><NN><CD>}
                    {<FW><JJ><NNS><CD>}
                    {<FW><NNS><NN><CD>}
                    {<FW><NN><NNS><CD>}
                    {<FW><NNS><NN><CD>}
                    {<JJR><NNS><NN><CD>}
                    {<JJR><NNS><NNS><CD>}
                    {<JJR><NN><NNS><CD>}
                    {<JJR><NN><NN><CD>}
                    {<JJR><NNS><NNS><CD>}
                    {<JJR><NN><NN><CD>}
                    {<JJR><NNS><NN><CD>}
                    {<JJR><NN><NNS><CD>}
                    {<JJR><JJ><NN><CD>}
                    {<JJR><JJ><NNS><CD>}
                    {<JJ><JJ><NNS><CD>}
                    {<JJ><JJ><NN><CD>}
                    {<NNS><NN><NN><CD>}
                    {<NNS><RBR><NNS><CD>}
                    {<NN><RBR><NN><CD>}
                    {<NNS><RBR><NN><CD>}
                    {<NN><RBR><NNS><CD>}
                    {<NN><JJ><NNS><CD>}
                    {<NN><JJ><NN><CD>}
                    {<NN><NNS><NNS><CD>}
                    {<NN><NNS><NN><CD>}
                    {<NNS><NN><NN><CD>}
                    {<RBR><NNS><NN><CD>}
                    {<RBR><NN><NN><CD>}
                    {<RBR><NNS><NNS><CD>}
                    {<RBR><NN><NNS><CD>}
                    {<CD><JJ><JJ>}
                    {<CD><NNS><JJ>}
                    {<CD><NN><JJ>}
                    {<CD><NN><VBP>}
                    {<CD><NNS><VBP>}  
                    {<CD><VBP><JJ>}
                    {<CD><JJ><NN>}
                    {<CD><JJ><NNS>}
                    {<CD><VBZ><NN>}
                    {<CD><VBZ><NN>}
                    {<CD><VB><NN>}
                    {<CD><VBP><NN>}
                    {<CD><VBP><NNS>}
                    {<CD><VBZ><JJ>}
                    {<CD><VBP><JJR>}
                    {<CD><VBP><FW>}
                    {<CD><VBR><NN>}
                    {<CD><RBR><NN>}
                    {<CD><JJR><NNS>}
                    {<CD><JJR><NN>}
                    {<CD><FW><NNS>}
                    {<CD><FW><NN>}
                    {<CD><NN><NN>}
                    {<CD><NNS><NN>}
                    {<CD><NN><NNS>}
                    {<CD><NN><NN>}
                    {<CD><NNS><NN>}
                    {<CD><NN><NNS>}
                    {<CD><NNS><NNS>}
                    {<CD><NNS><RBR>}
                    {<CD><NN><RBR>}
                    {<CD><NNS><RBR>}
                    {<CD><RBR><NN>}
                    {<CD><RBR><NNS>}
                    {<NN><NNS><CD>}
                    {<NN><NN><CD>}
                    {<NNS><NN><CD>}
                    {<NN><NNS><CD>}
                    {<VBZ><JJ><CD>}
                    {<VBP><JJR><CD>}
                    {<VBP><FW><CD>}
                    {<VBR><NN><CD>}
                    {<RBR><NN><CD>}
                    {<JJR><NNS><CD>}
                    {<JJR><NN><CD>}
                    {<FW><NNS><CD>}
                    {<FW><NN><CD>}
                    {<NN><NN><CD>}
                    {<NNS><NN><CD>}
                    {<NN><VBD><CD>}
                    {<NNS><VBD><CD>}
                    {<VB><NN><CD>}
                    {<NNS><RBR><CD>}
                    {<NN><RBR><CD>}
                    {<NNS><RBR><CD>}
                    {<RBR><NN><CD>}
                    {<RBR><NNS><CD>}     
                    {<CD><NN>}
                    {<CD><NNS>}
                    {<CD><VBZ>}
                    {<CD><VBP>}
                    {<CD><JJ>}
                    {<RBR><CD>}
                    {<NN><CD>}
                    {<NNS><CD>}
                    {<VBZ><CD>}
                    {<VBP><CD>}
                    {<JJ><CD>}
                    {<RBR><CD>}
                    {<RB><CD>}



            """

        NPChunker = nltk.RegexpParser(patternz)

        def prepare_text(input):
            sentences = nltk.sent_tokenize(input)
            sentences = [nltk.word_tokenize(sent) for sent in sentences] 
            sentences = [nltk.pos_tag(sent) for sent in sentences]
            sentences = [NPChunker.parse(sent) for sent in sentences]
            return sentences


        def parsed_text_to_NP(sentences):
            nps = []
            for sent in sentences:
                tree = NPChunker.parse(sent)
               # print(tree)
                for subtree in tree.subtrees():
                    if subtree.label() == 'NP':
                        t = subtree
                        t = ' '.join(word for word, tag in t.leaves())
                        nps.append(t)
            return nps


        def sent_parse(input):
            sentences = prepare_text(input)
            nps = parsed_text_to_NP(sentences)
            return nps

        poso1=    sent_parse(itemssentance)       # We get a list of all numbers in the text

        sems=[]
        for i22 in poso1:
            ale=i22.split(' ')
            #print(ale)
            for i33 in ale:
                #print(i33)
                if i33 in numberslis:
                    #print(i33)
                    ale.remove(i33)
                  #  print(ale)
                    sems.append(ale)
               # else:
               #     items.append(i33)

        items=[]
        for ij in sems:
            iafs=' '.join(ij)
            items.append(iafs)  


        ###################################################################################################


        items_list=items
        #print(numberslist)
        #print(items_list)
        #print(mylist)
        #full_list = np.array((items_list, numberslist, mylist))


        ########################We now merge and convert the 3 lists to a csv file ############################################################################

        full_list=[]
        full_list.append(items_list)
        full_list.append(numberslist)
        full_list.append(mylist)


        full_list=zip(*full_list)
        print(full_list)

        my_df = pd.DataFrame(full_list)
        my_df.columns =['items','quantity','unit']
        return my_df
transcriber()

,items,quantity,unit
0,rice,5.0,kilogram
1,chips,1.0,--
2,pepsi,1,--
3,candies,100.0,--
